In [1]:
import requests
import scaleapi
import json
from os import environ


Retrieving Project - testing API key

In [2]:
project_api_url = "https://api.scale.com/v1/projects/Traffic Sign Detection"

headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", project_api_url, headers=headers)
pretty_json = json.loads(response.text)
print (json.dumps(pretty_json, indent=4))


{
    "type": "annotation",
    "pipelineName": "standard_task",
    "created_at": "2020-07-18T01:20:22.280Z",
    "param_history": [
        {
            "created_at": "2020-12-17T23:18:07.540Z",
            "version": 0,
            "patch": false
        }
    ],
    "name": "Traffic Sign Detection",
    "pinned": true,
    "archived": false,
    "customGradersUsedOrDismissed": false,
    "edgeCaseAlertsEnabled": true,
    "turnOnLongHints": true,
    "isAudioTranscription": false,
    "useLabelerManagementPlatform": false,
    "containsAdultContent": false,
    "useTextCollectionTemplate": false,
    "datasetLinks": [],
    "taxonomy": {
        "patch": false
    }
}


Retrieving Tasks

In [3]:
task_retrieve_url = "https://api.scale.com/v1/tasks?status=completed&project=Traffic Sign Detection&limit=100"
task_detail_url = "https://api.scale.com/v1/task/"

headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", task_retrieve_url, headers=headers)

pretty_response = json.loads(response.text)

# list of tasks
tasks = pretty_response["docs"] 

for i in tasks:
    print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", task_detail_url+i["task_id"], headers=headers)

    pretty_response_task = json.loads(response_task.text)

    print (json.dumps(pretty_response_task, indent=4))

"5f127f6f26831d0010e985e5"
{
    "task_id": "5f127f6f26831d0010e985e5",
    "created_at": "2020-07-18T04:49:51.129Z",
    "completed_at": "2020-07-18T05:03:26.917Z",
    "type": "annotation",
    "status": "completed",
    "instruction": "Draw a tight box around each object.",
    "params": {
        "attachment": "https://observesign.s3-us-west-2.amazonaws.com/traffic_sign_1.jpg",
        "attachment_type": "image",
        "objects_to_annotate": [
            "traffic_control_sign",
            "construction_sign",
            "information_sign",
            "policy_sign",
            "non_visible_face"
        ],
        "with_labels": true,
        "min_width": 0,
        "min_height": 0,
        "examples": [],
        "annotation_attributes": {
            "occlusion": {
                "description": "What percent of the object is occluded?",
                "choices": [
                    "0%",
                    "25%",
                    "50%",
                    "75%",
  

# Phase 2: one QC

In [32]:
test_task_id = "5f127f5f3a6b100017232099"
headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", task_detail_url+test_task_id, headers=headers)

pretty_response = json.loads(response.text)

print (json.dumps(pretty_response['response']['annotations'][29], indent=4))
if pretty_response['response']['annotations'][29]['width']*pretty_response['response']['annotations'][29]['height'] > 0.2*1396*850:
    if pretty_response['response']['annotations'][29]['label'] == 'non_visible_face':
        if pretty_response['response']['annotations'][29]['attributes']['background_color'] == 'not_applicable':
            print('ERROR: something is wrong in {} tag ', pretty_response['response']['annotations'][29]['uuid'])
else:
    print ('False')
# if (pretty_response_annotations[29]['width']*pretty_response_annotations[29]['height'] < 

{
    "label": "non_visible_face",
    "attributes": {
        "occlusion": "0%",
        "truncation": "0%",
        "background_color": "not_applicable"
    },
    "uuid": "25e6e90c-7acb-4786-a0fb-ca7b10fc3687",
    "width": 518,
    "height": 654,
    "geometry": "box",
    "left": 2,
    "top": 196
}
ERROR: something is wrong in {} tag  25e6e90c-7acb-4786-a0fb-ca7b10fc3687
